In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Red Wine Quality Prediction using Random Forest
### Loading Dataset and Preparing Dataset
#### 1. Reading File

In [ ]:
winequality = pd.read_csv("/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")
winequality.head()

#### 2. Splitting into X and y

In [ ]:
X=winequality.drop("quality",axis=1)
y=winequality["quality"]

#### 3. Splitting into Train and Test Data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

### Fitting into Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train,y_train)

In [ ]:
model.score(X_test,y_test)

### Prediction

In [ ]:
y_preds = model.predict(X_test)
y_preds[:20]

### Classification Report and Confusion Matrix

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(classification_report(y_test,y_preds))

In [ ]:
conf_matrix = confusion_matrix(y_test,y_preds)
print(conf_matrix)

In [ ]:
accuracy_score(y_test,y_preds)

### Prediction Probability

In [ ]:
model.predict_proba(X_test[:5])

In [ ]:
np.random.seed(42)
from sklearn.model_selection import cross_val_score
for est in range(10,200,10):
    print(f"Trying with {est} estimators:")
    model = RandomForestClassifier(n_estimators= est).fit(X_train,y_train)
    score = model.score(X_test,y_test)*100
    print(f"Model Accuracy Score: {score}%")
    crossvalscore = np.mean(cross_val_score(model,X,y,cv=5)) *100
    print(f"Cross-Validation Score: {crossvalscore}%\n")

### Finding Best Estimator

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators': [i for i in range(10,200,10)]}
grid = GridSearchCV(RandomForestClassifier(),
                   param_grid,
                   cv=5)
grid.fit(X,y)

grid.best_params_

In [ ]:
model = grid.best_estimator_
model = model.fit(X_train,y_train)

model.score(X_test,y_test)

### Save Model and Reload Model

In [ ]:
import pickle
pickle.dump(model,open("RedWineModel.pkl","wb"))

In [ ]:
loadmodel = pickle.load(open("RedWineModel.pkl","rb"))
loadmodel.score(X_test,y_test)